In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, match_incomplete_to_complete_data

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch, \
                                           plot_interpolations

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.autopredictors.scripts.virt_scan_data import load_partial_pointclouds

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
n_pc_samples = 2048
do_training = False
do_plotting = True
first_time_running = False
load_model = True
seed = 42
GPU = 0
max_training_epochs = 1000
loss = 'chamfer'
experiment_name = '1_angela_train_3dec_z_rotate_and_spn'

In [4]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA'
partial_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Partial_PCs/Angela_synthetic', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/partial_clouds')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/partial_clouds/1_angela_train_3dec_z_rotate_and_spn'

In [5]:
train_list = '/orions4-zfs/projects/lins2/Panos_Space/DATA/From_Matthias/train_test_splits/train_models_with_scan_id.txt'
pclouds, model_ids, syn_ids = load_partial_pointclouds(train_list, partial_pclouds_path, 50)

153540 partial point clouds were loaded.


/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/tf_lab/point_clouds/in_out.py:56: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


In [6]:
model_names = np.array([i[0] for i in model_ids], dtype=object)
scan_ids = np.array([i[1] for i in model_ids], dtype=object)
model_unames = np.array(model_names + '.' + scan_ids + '.' + syn_ids, dtype=object)
train_data = PointCloudDataSet(pclouds, labels=model_unames)

In [7]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:
    n_input = [n_pc_samples, 3] 
    
    encoder_args = {'spn': True}
    
    decoder_args = {'layer_sizes': [512, 1024, np.prod(n_input)], 
                    'non_linearity': tf.nn.relu
                   }

    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = True,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args,
                encoder_args = encoder_args
               )

    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'non_linearity': <function relu at 0x7feff80bde60>, 'layer_sizes': [512, 1024, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'spn': True}
               experiment_name: 1_angela_train_3dec_z_rotate_and_spn
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/partial_clouds/1_a

In [8]:
if do_training:
    training_stats = []
    training_stats.append(ae.train(train_data, conf))    
    with open(osp.join(conf.train_dir, 'train_stats.txt'), 'a') as fout:
        np.savetxt(fout, np.array(training_stats)[0])

In [13]:
# TSNE PLOTTING
from tf_lab.autopredictors.scripts.virt_scan_data import plotting_default_params
from tf_lab.autopredictors.scripts.virt_scan_data import pc_loader as vscan_loader
from tf_lab.autopredictors.scripts.helper import shape_net_core_synth_id_to_category
from general_tools.clustering.plt import plot_2d_embedding_in_grid_greedy_way, plot_2d_embedding_in_grid_forceful

top_image_dir = osp.join(partial_pclouds_path, 'images')

samples_per_class = 200
selected = []
for syn_id in np.unique(syn_ids):
    in_class = np.where(syn_ids == syn_id)[0]
    selected += np.ndarray.tolist(np.random.choice(in_class, samples_per_class, replace=False))

plt.ioff()    
for i in selected:
    in_f = osp.join(partial_pclouds_path, syn_ids[i], model_names[i] + scan_ids[i] + '.ply')
    points = vscan_loader(in_f)[0]
    category_i = shape_net_core_synth_id_to_category[syn_ids[i]]
    params = plotting_default_params(category_i)
    save_name = syn_ids[i] + '.' + model_names[i] + '.' + scan_ids[i] + '.png'
    fig = Point_Cloud(points=points).plot(show=False, show_axis=False, **params);
    fig.savefig(osp.join(top_image_dir, save_name))
    plt.close()

In [14]:
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset
from sklearn.manifold import TSNE
tsne_dataset = PointCloudDataSet(pclouds[selected], labels=model_unames[selected], init_shuffle=False)
feed, latent_codes, ids = latent_embedding_of_entire_dataset(tsne_dataset, ae, conf)
model = TSNE(n_components=2, random_state=seed, init='pca', verbose=True)
tsne_small = model.fit_transform(latent_codes)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 1600
[t-SNE] Computed conditional probabilities for sample 1600 / 1600
[t-SNE] Mean sigma: 2.233930
[t-SNE] KL divergence after 75 iterations with early exaggeration: 0.717674
[t-SNE] Error after 100 iterations: 0.717674


In [15]:
# Make a list with the file-names of all to be used images.
image_files = []
for i in selected:
    save_name = syn_ids[i] + '.' + model_names[i] + '.' + scan_ids[i] + '.png'
    image_files.append(osp.join(top_image_dir, save_name))

In [16]:
plot_2d_embedding_in_grid_forceful(tsne_small, image_files, big_dim=8000, small_dim=200, save_file='tsne_incomplete_ae.png');
# plot_2d_embedding_in_grid_greedy_way(tsne_small, image_files, big_dim=8000, small_dim=200, save_file='tsne_partial_ae.png');